In [1]:
import tensorflow as tf
tf.VERSION

'1.14.0'

In [2]:
import pandas as pd
import os
import tensorflow as tf
# import cv2 as cv
import numpy as np
import glob
# # from random import shuffle
#from google.colab import drive
import matplotlib.pyplot as plt
import csv
import re
import json
import math
from sklearn.utils import shuffle
import tensorflow_hub as hub
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Input, Flatten, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Bidirectional,LSTM,GlobalMaxPool1D,Dense,Concatenate
from keras.utils.vis_utils import plot_model
from keras import losses
from keras.layers.merge import Multiply,Dot
import keras.backend as K


Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('video_LLDs_FAUs/test_01_OpenFace2.1.0_Pose_gaze_AUs.csv')

dataset.columns

Index(['frame', 'timestamp', 'confidence', 'success', 'pose_Tx', 'pose_Ty',
       'pose_Tz', 'pose_Rx', 'pose_Ry', 'pose_Rz', 'gaze_0_x', 'gaze_0_y',
       'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x',
       'gaze_angle_y', 'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r',
       'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r',
       'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'AU01_c', 'AU02_c',
       'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c',
       'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c',
       'AU28_c', 'AU45_c'],
      dtype='object')

In [4]:
metadata = pd.read_csv('metadata.csv')
sample = pd.read_csv('video_LLDs_FAUs/training_001_OpenFace2.1.0_Pose_gaze_AUs.csv')
sample_bow = pd.read_csv('video_BoVW/development_01_BoVW_openFace_2.1.0_Pose_Gaze_AUs.csv', header = None, delimiter = ';')


In [5]:
bow_cols = sample_bow.columns[2:102]

In [6]:
path = 'video_BoVW/'
bow_train = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('train') and file.startswith(i):
      c+=1
#       print(i)
#       print(c)
      
#       print(file[0:4]=='trai'):
        
      data = pd.read_csv('video_BoVW/'+file, header = None, delimiter=';')
      data = data[bow_cols]
      bow_train.append(data)

pose_cols = sample.columns[4:10]
gaze_cols = sample.columns[10:18]
au_cols = sample.columns[18:]

In [7]:
path = 'video_LLDs_FAUs/'
pose_train = []
gaze_train = []
au_train = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('train') and file.startswith(i):
      c+=1
#       print(i)
#       print(c)
      
#       print(file[0:4]=='trai'):
        
      feat = pd.read_csv('video_LLDs_FAUs/'+file)
      feat = feat.iloc[::3, :]
      pose_train.append(feat[pose_cols])
      gaze_train.append(feat[gaze_cols])
      au_train.append(feat[au_cols])

In [8]:
path = 'video_BoVW/'
bow_dev = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('development') and file.startswith(i):
      c+=1
      #print(i)
     # print(c)
      
#       print(file[0:4]=='trai'):
        
      data = pd.read_csv('video_BoVW/'+file, header = None, delimiter=';')
      data = data[bow_cols]
      bow_dev.append(data)

In [9]:
path = 'video_LLDs_FAUs/'
pose_dev = []
gaze_dev = []
au_dev = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('dev') and file.startswith(i):
      c+=1
      print(i,c)
      
#       print(file[0:4]=='trai'):
        
      feat = pd.read_csv('video_LLDs_FAUs/'+file)
      feat = feat.iloc[::3, :]
      pose_dev.append(feat[pose_cols])
      gaze_dev.append(feat[gaze_cols])
      au_dev.append(feat[au_cols])

development_01 1
development_02 2
development_03 3
development_04 4
development_05 5
development_06 6
development_07 7
development_08 8
development_09 9
development_10 10
development_11 11
development_12 12
development_13 13
development_14 14
development_15 15
development_16 16
development_17 17
development_18 18
development_19 19
development_20 20
development_21 21
development_22 22
development_23 23
development_24 24
development_25 25
development_26 26
development_27 27
development_28 28
development_29 29
development_30 30
development_31 31
development_32 32
development_33 33
development_34 34
development_35 35
development_36 36
development_37 37
development_38 38
development_39 39
development_40 40
development_41 41
development_42 42
development_43 43
development_44 44
development_45 45
development_46 46
development_47 47
development_48 48
development_49 49
development_50 50
development_51 51
development_52 52
development_53 53
development_54 54
development_55 55
development_56 56


In [10]:
def send(t_data,e_data):
    temp=[]
    ret_data=[]
    for ind,r in enumerate(e_data):
#         if(ind>0):
        x=r[0].split(";")
#         a=x.pop(0)
#         b=x.pop(0)
#         print(a,b)
        x=x[2:]
        xx=list(map(np.float32,x))
#         print(len(xx))
        temp.append(xx)
#     print(len(temp))
#     m.append(len(temp))
    for ind , r  in enumerate(t_data):
#         print(type(r[0]),r[0],r)
        if(ind>0):
            start_time,stop_time=math.floor(float(r[0])),math.floor(float(r[1]))
    #             print(r[0],r[1])
            if(stop_time<len(temp) and (stop_time-start_time)<100):
                for i in range(int(start_time),int(stop_time)):
#                     print(i)
                    ret_data.append(temp[i])
    m.append(len(ret_data))
    return ret_data

In [11]:
def send_BOAW(t_data,e_data):
    temp=[]
    ret_data=[]
    for ind,r in enumerate(e_data):
#         if(ind>0):
        x=r[0].split(";")
#         x.pop(0)
#         x.pop(0)
        x=x[2:]
        xx=list(map(np.float32,x))
#         print(len(xx))
        temp.append(xx)
#     print(len(temp))
#     m.append(len(temp))
    for ind , r  in enumerate(t_data):
#         print(type(r[0]),r[0],r)
        if(ind>0):
            start_time,stop_time=float(r[0]),float(r[1])
    #             print(r[0],r[1])
            if(stop_time*10<len(temp) and (stop_time-start_time)<100):
                for i in range(int(start_time*10),int(stop_time*10)):
#                     print(i)
                    ret_data.append(temp[i])
    m.append(len(ret_data))
    return ret_data

In [12]:
target_mfcc=[]
m=[]
mfcc_data=[]
emb_dim_mfcc=1300
pth=["audio_functionnals_MFCCs/*.csv"]
for filename in glob.glob(pth[0]):#CHANGE PATH
#   print(filename)
    tmp=[]
    if 'test' not in filename:
        print(filename)
        target_mfcc.append(filename.split("/")[-1].split(".")[0][:-11])
    #     print(target)
        with open(filename,"rt") as f:
            b_data=csv.reader(f)
#             for filename1 in glob.glob("/content/drive/My Drive/AVEC_trans/*"+target_mfcc[-1]+"*.csv"):
            filename1="ASR_transcripts/"+target_mfcc[-1]+"_Transcript.csv"
            with open(filename1,"rt") as f1:
                    t_data=csv.reader(f1)
                    tmp=send(t_data,b_data)
#                 ege_data.append(tmp)
                    mf=np.asarray(tmp)
                    print(mf.shape)
                    a = np.zeros((emb_dim_mfcc-mf.shape[0],78))
                    mf=np.vstack((a,mf))
#                 m.append(mf.shape[0])
                    mfcc_data.append(mf)
        print(mf.shape)

audio_functionnals_MFCCs/training_068_OpenSMILE2.3.0_mfcc.csv
(204, 78)
(1300, 78)
audio_functionnals_MFCCs/development_46_OpenSMILE2.3.0_mfcc.csv
(297, 78)
(1300, 78)
audio_functionnals_MFCCs/development_04_OpenSMILE2.3.0_mfcc.csv
(231, 78)
(1300, 78)
audio_functionnals_MFCCs/training_134_OpenSMILE2.3.0_mfcc.csv
(652, 78)
(1300, 78)
audio_functionnals_MFCCs/training_009_OpenSMILE2.3.0_mfcc.csv
(214, 78)
(1300, 78)
audio_functionnals_MFCCs/training_122_OpenSMILE2.3.0_mfcc.csv
(388, 78)
(1300, 78)
audio_functionnals_MFCCs/training_051_OpenSMILE2.3.0_mfcc.csv
(1288, 78)
(1300, 78)
audio_functionnals_MFCCs/development_53_OpenSMILE2.3.0_mfcc.csv
(173, 78)
(1300, 78)
audio_functionnals_MFCCs/training_039_OpenSMILE2.3.0_mfcc.csv
(527, 78)
(1300, 78)
audio_functionnals_MFCCs/development_17_OpenSMILE2.3.0_mfcc.csv
(769, 78)
(1300, 78)
audio_functionnals_MFCCs/training_075_OpenSMILE2.3.0_mfcc.csv
(483, 78)
(1300, 78)
audio_functionnals_MFCCs/training_001_OpenSMILE2.3.0_mfcc.csv
(221, 78)
(1300,

(615, 78)
(1300, 78)
audio_functionnals_MFCCs/training_111_OpenSMILE2.3.0_mfcc.csv
(465, 78)
(1300, 78)
audio_functionnals_MFCCs/development_45_OpenSMILE2.3.0_mfcc.csv
(569, 78)
(1300, 78)
audio_functionnals_MFCCs/training_056_OpenSMILE2.3.0_mfcc.csv
(853, 78)
(1300, 78)
audio_functionnals_MFCCs/training_093_OpenSMILE2.3.0_mfcc.csv
(411, 78)
(1300, 78)
audio_functionnals_MFCCs/training_046_OpenSMILE2.3.0_mfcc.csv
(617, 78)
(1300, 78)
audio_functionnals_MFCCs/training_092_OpenSMILE2.3.0_mfcc.csv
(412, 78)
(1300, 78)
audio_functionnals_MFCCs/training_015_OpenSMILE2.3.0_mfcc.csv
(251, 78)
(1300, 78)
audio_functionnals_MFCCs/training_012_OpenSMILE2.3.0_mfcc.csv
(1090, 78)
(1300, 78)
audio_functionnals_MFCCs/training_008_OpenSMILE2.3.0_mfcc.csv
(298, 78)
(1300, 78)
audio_functionnals_MFCCs/training_076_OpenSMILE2.3.0_mfcc.csv
(372, 78)
(1300, 78)
audio_functionnals_MFCCs/training_144_OpenSMILE2.3.0_mfcc.csv
(143, 78)
(1300, 78)
audio_functionnals_MFCCs/training_049_OpenSMILE2.3.0_mfcc.csv
(

(447, 78)
(1300, 78)
audio_functionnals_MFCCs/training_151_OpenSMILE2.3.0_mfcc.csv
(135, 78)
(1300, 78)
audio_functionnals_MFCCs/training_137_OpenSMILE2.3.0_mfcc.csv
(211, 78)
(1300, 78)
audio_functionnals_MFCCs/training_070_OpenSMILE2.3.0_mfcc.csv
(567, 78)
(1300, 78)
audio_functionnals_MFCCs/training_035_OpenSMILE2.3.0_mfcc.csv
(699, 78)
(1300, 78)
audio_functionnals_MFCCs/development_29_OpenSMILE2.3.0_mfcc.csv
(350, 78)
(1300, 78)
audio_functionnals_MFCCs/training_138_OpenSMILE2.3.0_mfcc.csv
(689, 78)
(1300, 78)
audio_functionnals_MFCCs/development_03_OpenSMILE2.3.0_mfcc.csv
(619, 78)
(1300, 78)
audio_functionnals_MFCCs/training_047_OpenSMILE2.3.0_mfcc.csv
(216, 78)
(1300, 78)
audio_functionnals_MFCCs/development_15_OpenSMILE2.3.0_mfcc.csv
(305, 78)
(1300, 78)
audio_functionnals_MFCCs/training_073_OpenSMILE2.3.0_mfcc.csv
(473, 78)
(1300, 78)
audio_functionnals_MFCCs/training_013_OpenSMILE2.3.0_mfcc.csv
(522, 78)
(1300, 78)
audio_functionnals_MFCCs/training_109_OpenSMILE2.3.0_mfcc.cs

In [13]:
target_ege=[]
m=[]
ege_data=[]
emb_dim_ege=1410
pth=["audio_functionnals_eGeMAPS/*.csv"]
for filename in glob.glob(pth[0]):#CHANGE PATH
#   print(filename)
    tmp=[]
    if 'test' not in filename:
        print(filename)
        target_ege.append(filename.split("/")[-1].split(".")[0][:-11])
    #     print(target)
        with open(filename,"rt") as f:
            b_data=csv.reader(f)
#             for filename1 in glob.glob("/content/drive/My Drive/AVEC_trans/*"+target_ege[-1]+"*.csv"):
#                 print(filename1)
            filename1="ASR_transcripts/"+target_ege[-1]+"_Transcript.csv"
            with open(filename1,"rt") as f1:
                    t_data=csv.reader(f1)
                    tmp=send(t_data,b_data)
#                 ege_data.append(tmp)
                    mf=np.asarray(tmp)
                    print(mf.shape)
                    a = np.zeros((emb_dim_ege-mf.shape[0],88))
                    mf=np.vstack((a,mf))
#                 m.append(mf.shape[0])
                    ege_data.append(mf)
        print(mf.shape)

audio_functionnals_eGeMAPS/development_06_OpenSMILE2.3.0_egemaps.csv
(313, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_119_OpenSMILE2.3.0_egemaps.csv
(381, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_072_OpenSMILE2.3.0_egemaps.csv
(243, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_125_OpenSMILE2.3.0_egemaps.csv
(365, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_51_OpenSMILE2.3.0_egemaps.csv
(461, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_52_OpenSMILE2.3.0_egemaps.csv
(261, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_137_OpenSMILE2.3.0_egemaps.csv
(211, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_131_OpenSMILE2.3.0_egemaps.csv
(203, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_031_OpenSMILE2.3.0_egemaps.csv
(444, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_11_OpenSMILE2.3.0_egemaps.csv
(653, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_35_OpenSMILE2.3.0_egemaps.csv
(861, 88)
(1410, 88)
audio_functionnals_eGe

(134, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_091_OpenSMILE2.3.0_egemaps.csv
(647, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_155_OpenSMILE2.3.0_egemaps.csv
(367, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_163_OpenSMILE2.3.0_egemaps.csv
(314, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_075_OpenSMILE2.3.0_egemaps.csv
(481, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_020_OpenSMILE2.3.0_egemaps.csv
(524, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_159_OpenSMILE2.3.0_egemaps.csv
(547, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_065_OpenSMILE2.3.0_egemaps.csv
(491, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_058_OpenSMILE2.3.0_egemaps.csv
(176, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_143_OpenSMILE2.3.0_egemaps.csv
(541, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_064_OpenSMILE2.3.0_egemaps.csv
(898, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_43_OpenSMILE2.3.0_egemaps.csv
(250, 88)
(1410, 88)
audio_fun

(1410, 88)
audio_functionnals_eGeMAPS/development_16_OpenSMILE2.3.0_egemaps.csv
(949, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_19_OpenSMILE2.3.0_egemaps.csv
(359, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_112_OpenSMILE2.3.0_egemaps.csv
(777, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_21_OpenSMILE2.3.0_egemaps.csv
(222, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_149_OpenSMILE2.3.0_egemaps.csv
(1145, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_096_OpenSMILE2.3.0_egemaps.csv
(425, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_003_OpenSMILE2.3.0_egemaps.csv
(364, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_023_OpenSMILE2.3.0_egemaps.csv
(677, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_45_OpenSMILE2.3.0_egemaps.csv
(569, 88)
(1410, 88)
audio_functionnals_eGeMAPS/development_22_OpenSMILE2.3.0_egemaps.csv
(455, 88)
(1410, 88)
audio_functionnals_eGeMAPS/training_081_OpenSMILE2.3.0_egemaps.csv
(355, 88)
(1410, 88)
audio_func

In [14]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [15]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"i've","i have",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"that ' s","that is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"\'ll"," will",text) 
    text = re.sub(r"\'ve"," have",text)
    text = re.sub(r"\'re"," are",text)
    text = re.sub(r"\'d"," would",text)  
    text = re.sub(r"won't", "will not",text)
    text = re.sub(r"don't", "do not",text)
    text = re.sub(r"can't","can not", text)
    text = re.sub(r"hadn't","had not", text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"wouldn't","would not", text)
    text = re.sub(r"weren't","were not", text)
    text = re.sub(r"shouldn't","should not", text)
    text = re.sub(r"doesn't","does not", text)
    text = re.sub(r"couldn't","could not", text)
    text = re.sub(r"isn't","is not", text)
    text = re.sub(r"hasn't","has not", text)
    text = re.sub(r"wasn't","was not", text)
    text = re.sub(r"haven't","have not", text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"wouldnt'","would not", text)
#     text = re.sub(r"shouldnt'","should not", text)
#     text = re.sub(r"doesnt'","does not", text)
#     text = re.sub(r"couldnt'","could not", text)
#     text = re.sub(r"isnt'","is not", text)
#     text = re.sub(r"wasn't","was not", text)
#     text = re.sub(r"havent'","have not", text)
    text = re.sub(r"aren't","are not", text)
#     text = re.sub(r"werent","were not", text)
    text = re.sub(r" em "," them ", text)
    text = re.sub(r" there's "," there is ", text)
    text = re.sub(r"let's","let us", text)
    text = re.sub(r" who's "," who is ", text)
    text = re.sub(r"\'s","", text)
    text = re.sub(r"'","", text)
# #     text = re.sub(r"arent","are not", text)
#     text = re.sub(r"  "," ", text)
#     text = re.sub(r"[-?!@#$%^&*(~,;)+=_`<>{}.\"']","",text)
#     pattern = r'[^a-zA-z0-9\s]'
#     text = re.sub(pattern, '', text)
    return text

In [16]:
def sent(data,cnt):
#   Participant_transcript=[rows[0].split("\t") for rows in data if len(rows)>0 if rows[0].split("\t")[2] == "Participant" ]
    if cnt==0:
      Participant_transcript=[rows[0].split("\t") for rows in data if len(rows)>0 if rows[0].split("\t")[2] == "Participant" ]
#   print(Participant_transcript)
      words=[clean_text(x[3].strip()).strip() for x in Participant_transcript]
    else:
      words=[clean_text(x[2].strip()).strip() for x in data]
      words.pop(0)
#     print(words)
    for ind,val in enumerate(words):
        a=val.split(' ')
        for ii,tp in enumerate(a):
            if(len(tp)):
                if(tp[0]=='<'and tp[-1]=='>'):
                    a[ii]=tp[1:-1]
                elif(tp[0]=='<' ):
                    a[ii]=tp[1:]
        words[ind]=' '.join(a).strip()        
    m.append(len(words))
#     print(words)
    return words

In [17]:
tf.VERSION
sentences=[]
target_asr=[]
m=[]
embed_fn = embed_useT('USE/')
embedding_matrix=[]

W0715 10:57:21.154205 140605471172352 deprecation.py:323] From /home/siddharth/my_env/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
emb_dim_asr=400
pth=["ASR_transcripts/*.csv"]
for cnt,p in enumerate(pth):
  cnt+=1
  if cnt>0:
    for filename in glob.glob(p):#CHANGE PATH
        print(filename)
        if 'test' not in filename:
          if(cnt==0):
            target_asr.append(filename.split("/")[-1].split("_")[0])
          else:
            target_asr.append(filename.split("/")[-1].split(".")[0][:-11])
          with open(filename,"rt") as f:
            data=csv.reader(f)
            msg_txt=sent(data,cnt)
      #       print(msg_txt)  
            emb=embed_fn(msg_txt)
            a = np.zeros((emb_dim_asr-emb.shape[0], 512))
            x=np.vstack((a,emb))
            embedding_matrix.append(x)
            print(emb.shape)

ASR_transcripts/development_43_Transcript.csv
(106, 512)
ASR_transcripts/training_040_Transcript.csv
(64, 512)
ASR_transcripts/test_40_Transcript.csv
ASR_transcripts/training_039_Transcript.csv
(44, 512)
ASR_transcripts/test_21_Transcript.csv
ASR_transcripts/test_25_Transcript.csv
ASR_transcripts/training_062_Transcript.csv
(57, 512)
ASR_transcripts/test_34_Transcript.csv
ASR_transcripts/training_089_Transcript.csv
(80, 512)
ASR_transcripts/training_049_Transcript.csv
(64, 512)
ASR_transcripts/development_25_Transcript.csv
(91, 512)
ASR_transcripts/training_104_Transcript.csv
(47, 512)
ASR_transcripts/training_135_Transcript.csv
(91, 512)
ASR_transcripts/development_34_Transcript.csv
(70, 512)
ASR_transcripts/test_18_Transcript.csv
ASR_transcripts/training_128_Transcript.csv
(64, 512)
ASR_transcripts/training_080_Transcript.csv
(165, 512)
ASR_transcripts/training_098_Transcript.csv
(73, 512)
ASR_transcripts/development_56_Transcript.csv
(111, 512)
ASR_transcripts/training_063_Transcrip

(74, 512)
ASR_transcripts/training_053_Transcript.csv
(131, 512)
ASR_transcripts/development_15_Transcript.csv
(117, 512)
ASR_transcripts/training_078_Transcript.csv
(65, 512)
ASR_transcripts/test_45_Transcript.csv
ASR_transcripts/test_11_Transcript.csv
ASR_transcripts/development_19_Transcript.csv
(107, 512)
ASR_transcripts/training_116_Transcript.csv
(71, 512)
ASR_transcripts/training_079_Transcript.csv
(164, 512)
ASR_transcripts/development_07_Transcript.csv
(69, 512)
ASR_transcripts/development_41_Transcript.csv
(60, 512)
ASR_transcripts/training_004_Transcript.csv
(203, 512)
ASR_transcripts/development_29_Transcript.csv
(82, 512)
ASR_transcripts/development_18_Transcript.csv
(141, 512)
ASR_transcripts/training_120_Transcript.csv
(127, 512)
ASR_transcripts/test_09_Transcript.csv
ASR_transcripts/training_086_Transcript.csv
(92, 512)
ASR_transcripts/training_074_Transcript.csv
(75, 512)
ASR_transcripts/training_093_Transcript.csv
(90, 512)
ASR_transcripts/training_092_Transcript.csv


In [19]:
target_BOAW_ege=[]
m=[]
BOAW_data_ege=[]
emb_dim_BOAW_ege=14050
pth=["audio_BoAW/*eGeMAP*.csv"]
for filename in glob.glob(pth[0]):#CHANGE PATH
#   print(filename)
    tmp=[]
    if 'test' not in filename:
        print(filename)
        target_BOAW_ege.append(filename.split("/")[-1].split(".")[0][:-17])
    #     print(target)
        with open(filename,"rt") as f:
            b_data=csv.reader(f)
#             for filename1 in glob.glob("/content/drive/My Drive/AVEC_trans/*"+target_BOAW_ege[-1]+"*.csv"):
#                 print(filename1)
            filename1="ASR_transcripts/"+target_BOAW_ege[-1]+"_Transcript.csv"
            with open(filename1,"rt") as f1:
                    t_data=csv.reader(f1)
                    tmp=send_BOAW(t_data,b_data)
#                 ege_data.append(tmp)
                    mf=np.asarray(tmp)
                    print(mf.shape)
                    a = np.zeros((emb_dim_BOAW_ege-mf.shape[0],100))
                    mf=np.vstack((a,mf))
#                 m.append(mf.shape[0])
                    BOAW_data_ege.append(mf)
        print(mf.shape)

audio_BoAW/training_018_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4173, 100)
(14050, 100)
audio_BoAW/training_132_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(8296, 100)
(14050, 100)
audio_BoAW/training_006_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(3695, 100)
(14050, 100)
audio_BoAW/development_22_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4541, 100)
(14050, 100)
audio_BoAW/development_04_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(2340, 100)
(14050, 100)
audio_BoAW/development_16_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(9492, 100)
(14050, 100)
audio_BoAW/development_38_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4311, 100)
(14050, 100)
audio_BoAW/development_23_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4630, 100)
(14050, 100)
audio_BoAW/training_148_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(5621, 100)
(14050, 100)
audio_BoAW/training_118_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(5803, 100)
(14050, 100)
audio_BoAW/training_024_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(3654, 100)
(14050, 100)
audio_BoAW/training_072_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(2580, 100)
(14050, 100)
audio_

(7184, 100)
(14050, 100)
audio_BoAW/training_124_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(10211, 100)
(14050, 100)
audio_BoAW/training_053_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(10690, 100)
(14050, 100)
audio_BoAW/training_117_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4639, 100)
(14050, 100)
audio_BoAW/development_31_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(5004, 100)
(14050, 100)
audio_BoAW/development_27_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4793, 100)
(14050, 100)
audio_BoAW/development_06_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(3116, 100)
(14050, 100)
audio_BoAW/training_017_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(5371, 100)
(14050, 100)
audio_BoAW/training_032_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(2043, 100)
(14050, 100)
audio_BoAW/training_158_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(1465, 100)
(14050, 100)
audio_BoAW/training_131_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(2020, 100)
(14050, 100)
audio_BoAW/training_045_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(2580, 100)
(14050, 100)
audio_BoAW/training_035_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(6995, 1

(1963, 100)
(14050, 100)
audio_BoAW/training_043_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(5915, 100)
(14050, 100)
audio_BoAW/training_151_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(1434, 100)
(14050, 100)
audio_BoAW/training_036_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(3281, 100)
(14050, 100)
audio_BoAW/training_076_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(3701, 100)
(14050, 100)
audio_BoAW/training_079_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4898, 100)
(14050, 100)
audio_BoAW/training_142_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(4352, 100)
(14050, 100)
audio_BoAW/training_108_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(9622, 100)
(14050, 100)
audio_BoAW/training_026_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(3685, 100)
(14050, 100)
audio_BoAW/development_20_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(1703, 100)
(14050, 100)
audio_BoAW/training_130_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(5253, 100)
(14050, 100)
audio_BoAW/training_054_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(10397, 100)
(14050, 100)
audio_BoAW/training_067_BoAW_openSMILE_2.3.0_eGeMAPS.csv
(7320, 100)
(

In [20]:
target_BOAW_mfcc=[]
m=[]
BOAW_data_mfcc=[]
emb_dim_BOAW_mfcc=14050
pth=["audio_BoAW/*MFCC*.csv"]
for filename in glob.glob(pth[0]):#CHANGE PATH
#   print(filename)
    tmp=[]
    if 'test' not in filename:
        print(filename)
        target_BOAW_mfcc.append(filename.split("/")[-1].split(".")[0][:-17])
    #     print(target)
        with open(filename,"rt") as f:
            b_data=csv.reader(f)
#             for filename1 in glob.glob("/content/drive/My Drive/AVEC_trans/*"+target_BOAW_mfcc[-1]+"*.csv"):
#                 print(filename1)
            filename1="ASR_transcripts/"+target_BOAW_mfcc[-1]+"_Transcript.csv"
            with open(filename1,"rt") as f1:
                    t_data=csv.reader(f1)
                    tmp=send_BOAW(t_data,b_data)
#                 ege_data.append(tmp)
                    mf=np.asarray(tmp)
                    print(mf.shape)
                    a = np.zeros((emb_dim_BOAW_mfcc-mf.shape[0],100))
                    mf=np.vstack((a,mf))
#                 m.append(mf.shape[0])
                    BOAW_data_mfcc.append(mf)
        print(mf.shape)

audio_BoAW/training_150_BoAW_openSMILE_2.3.0_MFCC.csv
(1867, 100)
(14050, 100)
audio_BoAW/training_105_BoAW_openSMILE_2.3.0_MFCC.csv
(9878, 100)
(14050, 100)
audio_BoAW/training_047_BoAW_openSMILE_2.3.0_MFCC.csv
(2183, 100)
(14050, 100)
audio_BoAW/training_113_BoAW_openSMILE_2.3.0_MFCC.csv
(7874, 100)
(14050, 100)
audio_BoAW/training_061_BoAW_openSMILE_2.3.0_MFCC.csv
(4321, 100)
(14050, 100)
audio_BoAW/development_21_BoAW_openSMILE_2.3.0_MFCC.csv
(2259, 100)
(14050, 100)
audio_BoAW/training_123_BoAW_openSMILE_2.3.0_MFCC.csv
(6587, 100)
(14050, 100)
audio_BoAW/training_017_BoAW_openSMILE_2.3.0_MFCC.csv
(5371, 100)
(14050, 100)
audio_BoAW/training_068_BoAW_openSMILE_2.3.0_MFCC.csv
(2104, 100)
(14050, 100)
audio_BoAW/training_005_BoAW_openSMILE_2.3.0_MFCC.csv
(9698, 100)
(14050, 100)
audio_BoAW/training_097_BoAW_openSMILE_2.3.0_MFCC.csv
(3576, 100)
(14050, 100)
audio_BoAW/training_071_BoAW_openSMILE_2.3.0_MFCC.csv
(2469, 100)
(14050, 100)
audio_BoAW/training_044_BoAW_openSMILE_2.3.0_MFCC.

(7181, 100)
(14050, 100)
audio_BoAW/training_131_BoAW_openSMILE_2.3.0_MFCC.csv
(2020, 100)
(14050, 100)
audio_BoAW/training_038_BoAW_openSMILE_2.3.0_MFCC.csv
(4304, 100)
(14050, 100)
audio_BoAW/training_045_BoAW_openSMILE_2.3.0_MFCC.csv
(2580, 100)
(14050, 100)
audio_BoAW/training_103_BoAW_openSMILE_2.3.0_MFCC.csv
(3654, 100)
(14050, 100)
audio_BoAW/development_43_BoAW_openSMILE_2.3.0_MFCC.csv
(2560, 100)
(14050, 100)
audio_BoAW/training_082_BoAW_openSMILE_2.3.0_MFCC.csv
(9204, 100)
(14050, 100)
audio_BoAW/development_03_BoAW_openSMILE_2.3.0_MFCC.csv
(6191, 100)
(14050, 100)
audio_BoAW/training_066_BoAW_openSMILE_2.3.0_MFCC.csv
(803, 100)
(14050, 100)
audio_BoAW/training_010_BoAW_openSMILE_2.3.0_MFCC.csv
(3404, 100)
(14050, 100)
audio_BoAW/training_055_BoAW_openSMILE_2.3.0_MFCC.csv
(7493, 100)
(14050, 100)
audio_BoAW/development_48_BoAW_openSMILE_2.3.0_MFCC.csv
(7745, 100)
(14050, 100)
audio_BoAW/development_05_BoAW_openSMILE_2.3.0_MFCC.csv
(2491, 100)
(14050, 100)
audio_BoAW/training_

(7737, 100)
(14050, 100)
audio_BoAW/training_122_BoAW_openSMILE_2.3.0_MFCC.csv
(3868, 100)
(14050, 100)
audio_BoAW/training_032_BoAW_openSMILE_2.3.0_MFCC.csv
(2043, 100)
(14050, 100)
audio_BoAW/training_030_BoAW_openSMILE_2.3.0_MFCC.csv
(2878, 100)
(14050, 100)
audio_BoAW/training_136_BoAW_openSMILE_2.3.0_MFCC.csv
(8535, 100)
(14050, 100)
audio_BoAW/training_092_BoAW_openSMILE_2.3.0_MFCC.csv
(4039, 100)
(14050, 100)
audio_BoAW/training_119_BoAW_openSMILE_2.3.0_MFCC.csv
(3846, 100)
(14050, 100)
audio_BoAW/training_016_BoAW_openSMILE_2.3.0_MFCC.csv
(2397, 100)
(14050, 100)
audio_BoAW/development_53_BoAW_openSMILE_2.3.0_MFCC.csv
(1687, 100)
(14050, 100)
audio_BoAW/training_086_BoAW_openSMILE_2.3.0_MFCC.csv
(5962, 100)
(14050, 100)
audio_BoAW/training_074_BoAW_openSMILE_2.3.0_MFCC.csv
(3897, 100)
(14050, 100)
audio_BoAW/development_01_BoAW_openSMILE_2.3.0_MFCC.csv
(1258, 100)
(14050, 100)
audio_BoAW/training_137_BoAW_openSMILE_2.3.0_MFCC.csv
(2124, 100)
(14050, 100)


In [21]:
BOAW_data_ege=np.asarray(BOAW_data_ege)

BOAW_data_mfcc=np.asarray(BOAW_data_mfcc)

embedding_matrix=np.asarray(embedding_matrix)

mfcc_data=np.asarray(mfcc_data)
ege_data=np.asarray(ege_data)

In [22]:
print(mfcc_data.shape)
print(ege_data.shape)
print(embedding_matrix.shape)
print(BOAW_data_ege.shape)
print(BOAW_data_mfcc.shape)

(219, 1300, 78)
(219, 1410, 88)
(220, 400, 512)
(219, 14050, 100)
(219, 14050, 100)


In [23]:
train_data_mfcc,train_data_ege,train_data_asr,train_data_BOAW_ege,train_data_BOAW_mfcc=[],[],[],[],[]
pth=["metadata.csv"]
for p in pth:
    with open(p,"rt") as f:
        l_data=csv.reader(f)
        for r in l_data:
          if len(r):
            if r[0] in target_mfcc and ('training' in r[0]):
#               print(r[0],r[3])
#               print(embedding_matrix[train_target.index(r[0])])
              train_data_mfcc.append(mfcc_data[target_mfcc.index(r[0])])
#               train_lbl.append(int(r[3]))
              train_data_ege.append(ege_data[target_ege.index(r[0])])
#               train_lbl_ege.append(int(r[3]))
              train_data_asr.append(embedding_matrix[target_asr.index(r[0])])
              train_data_BOAW_ege.append(BOAW_data_ege[target_BOAW_ege.index(r[0])])
              train_data_BOAW_mfcc.append(BOAW_data_mfcc[target_BOAW_mfcc.index(r[0])])

In [24]:
train_data_mfcc=np.asarray(train_data_mfcc,dtype=np.float32)
train_data_ege=np.asarray(train_data_ege,dtype=np.float32)
train_data_asr=np.asarray(train_data_asr,dtype=np.float32)
train_data_BOAW_mfcc=np.asarray(train_data_BOAW_mfcc,dtype=np.float32)
train_data_BOAW_ege=np.asarray(train_data_BOAW_ege,dtype=np.float32)


In [25]:
print(train_data_mfcc.shape)
# print(train_lbl_mfcc.shape)
print(train_data_ege.shape)
# print(train_lbl_ege.shape)
print(train_data_asr.shape)
print(train_data_BOAW_ege.shape)
print(train_data_BOAW_mfcc.shape)

(163, 1300, 78)
(163, 1410, 88)
(163, 400, 512)
(163, 14050, 100)
(163, 14050, 100)


In [26]:
val_data_mfcc,val_data_ege,val_data_asr,val_data_BOAW_ege,val_data_BOAW_mfcc=[],[],[],[],[]
pth=["metadata.csv"]
for p in pth:
    with open(p,"rt") as f:
        l_data=csv.reader(f)
        for r in l_data:
          if len(r):
            if r[0] in target_mfcc and ('development' in r[0]):
#               print(r[0],r[3])
#               print(embedding_matrix[train_target.index(r[0])])
              val_data_mfcc.append(mfcc_data[target_mfcc.index(r[0])])
#               val_lbl.append(int(r[3]))
              val_data_ege.append(ege_data[target_ege.index(r[0])])
#               val_lbl_ege.append(int(r[3]))
              val_data_asr.append(embedding_matrix[target_asr.index(r[0])])
              val_data_BOAW_ege.append(BOAW_data_ege[target_BOAW_ege.index(r[0])])
              val_data_BOAW_mfcc.append(BOAW_data_mfcc[target_BOAW_mfcc.index(r[0])])
#               val_lbl_asr.append(int(r[3]))

In [27]:
val_data_mfcc=np.asarray(val_data_mfcc,dtype=np.float32)
val_data_ege=np.asarray(val_data_ege,dtype=np.float32)
val_data_asr=np.asarray(val_data_asr,dtype=np.float32)
val_data_BOAW_mfcc=np.asarray(val_data_BOAW_mfcc,dtype=np.float32)
val_data_BOAW_ege=np.asarray(val_data_BOAW_ege,dtype=np.float32)
# val_lbl=n

In [28]:
print(val_data_mfcc.shape)
# print(val_lbl_mfcc.shape)
print(val_data_ege.shape)
# print(val_lbl_ege.shape)
print(val_data_asr.shape)
print(val_data_BOAW_ege.shape)
print(val_data_BOAW_mfcc.shape)

(56, 1300, 78)
(56, 1410, 88)
(56, 400, 512)
(56, 14050, 100)
(56, 14050, 100)


In [29]:
path = 'video_LLDs_FAUs/'
pose_test = [-1 for i in range(56)]
gaze_test = [-1 for i in range(56)]
au_test = [-1 for i in range(56)]
c = 0


for file in os.listdir(path):

    if file.startswith('test'):
        c+=1
        print(file,int(file[5:7]))

        #       print(file[0:4]=='trai'):

        feat = pd.read_csv('video_LLDs_FAUs/'+file)
        feat = feat.iloc[::3, :]
        pose_test[int(file[5:7])-1] = feat[pose_cols]
        gaze_test[int(file[5:7])-1] = feat[gaze_cols]
        au_test[int(file[5:7])-1] = feat[au_cols]



path = 'video_BoVW/'
bow_test = [-1 for i in range(56)]

c = 0


for file in os.listdir(path):

    if file.startswith('test'):
        c+=1
        print(file,int(file[5:7]))

        #       print(file[0:4]=='trai'):

        feat = pd.read_csv(path+file, header = None,delimiter = ';')
        bow_test[int(file[5:7])-1] = feat[bow_cols]

len(au_cols)

test_34_OpenFace2.1.0_Pose_gaze_AUs.csv 34
test_50_OpenFace2.1.0_Pose_gaze_AUs.csv 50
test_41_OpenFace2.1.0_Pose_gaze_AUs.csv 41
test_14_OpenFace2.1.0_Pose_gaze_AUs.csv 14
test_31_OpenFace2.1.0_Pose_gaze_AUs.csv 31
test_49_OpenFace2.1.0_Pose_gaze_AUs.csv 49
test_17_OpenFace2.1.0_Pose_gaze_AUs.csv 17
test_09_OpenFace2.1.0_Pose_gaze_AUs.csv 9
test_07_OpenFace2.1.0_Pose_gaze_AUs.csv 7
test_36_OpenFace2.1.0_Pose_gaze_AUs.csv 36
test_15_OpenFace2.1.0_Pose_gaze_AUs.csv 15
test_43_OpenFace2.1.0_Pose_gaze_AUs.csv 43
test_48_OpenFace2.1.0_Pose_gaze_AUs.csv 48
test_10_OpenFace2.1.0_Pose_gaze_AUs.csv 10
test_40_OpenFace2.1.0_Pose_gaze_AUs.csv 40
test_35_OpenFace2.1.0_Pose_gaze_AUs.csv 35
test_32_OpenFace2.1.0_Pose_gaze_AUs.csv 32
test_56_OpenFace2.1.0_Pose_gaze_AUs.csv 56
test_20_OpenFace2.1.0_Pose_gaze_AUs.csv 20
test_45_OpenFace2.1.0_Pose_gaze_AUs.csv 45
test_54_OpenFace2.1.0_Pose_gaze_AUs.csv 54
test_46_OpenFace2.1.0_Pose_gaze_AUs.csv 46
test_25_OpenFace2.1.0_Pose_gaze_AUs.csv 25
test_16_OpenF

35

In [30]:
target = []

for i,v in metadata.iterrows():
  if v['Participant_ID'].startswith('train'):
    target.append(v['PHQ_Score'])

dev_target = []

for i,v in metadata.iterrows():
  if v['Participant_ID'].startswith('dev'):
    dev_target.append(v['PHQ_Score'])

for i in pose_train:
#   print(i)
  i['pose_Tx'] = i['pose_Tx']/100
  i['pose_Ty'] = i['pose_Ty']/100
  i['pose_Tz'] = i['pose_Tz']/100

c = 0
for i in pose_test:
  print(c, end = ' ')
  c+=1
#   print(i)
  i['pose_Tx'] = i['pose_Tx']/100
  i['pose_Ty'] = i['pose_Ty']/100
  i['pose_Tz'] = i['pose_Tz']/100

c = 0
for i in pose_dev:
  print(c,end = ' ')
  c+=1
#   print(i)
  i['pose_Tx'] = i['pose_Tx']/100
  i['pose_Ty'] = i['pose_Ty']/100
  i['pose_Tz'] = i['pose_Tz']/100


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 

In [31]:
MAX_SEQ_LEN = 15000

train_seq_pose = pad_sequences(pose_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_pose = pad_sequences(pose_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_pose = pad_sequences(pose_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')

train_seq_gaze = pad_sequences(gaze_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_gaze = pad_sequences(gaze_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_gaze =  pad_sequences(gaze_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')

train_seq_au = pad_sequences(au_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_au = pad_sequences(au_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_au = pad_sequences(au_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')

train_seq_bow = pad_sequences(bow_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_bow = pad_sequences(bow_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_bow = pad_sequences(bow_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')


In [32]:
target = np.reshape(target,[-1,1])
dev_target = np.reshape(dev_target,[-1,1])

In [71]:
#NETWORK DEFINITION
tf.reset_default_graph()
print("network started")
true_targets = tf.placeholder(shape=(None,1),dtype = tf.float32)
lr = tf.placeholder(dtype=tf.float32)


input_bow = tf.placeholder(shape=(None,MAX_SEQ_LEN,100),dtype = tf.float32)
context_layer_bow = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_bow)

input_pose = tf.placeholder(shape=(None,MAX_SEQ_LEN,6),dtype = tf.float32)
context_layer_pose = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_pose)


input_gaze = tf.placeholder(shape=(None,MAX_SEQ_LEN,8),dtype = tf.float32, name='GAZE')
context_layer_gaze = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_gaze)


input_au = tf.placeholder(shape=(None,MAX_SEQ_LEN,35),dtype = tf.float32)
context_layer_au = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_au)


stacked_layer_visual = tf.stack([context_layer_bow,context_layer_pose, context_layer_gaze,context_layer_au], 
                                axis = 1)

concat_layer_visual = tf.keras.layers.Concatenate()([context_layer_bow, context_layer_pose, context_layer_gaze,
                                                     context_layer_au])

atten_dense_visual = tf.keras.layers.Dense(100)(concat_layer_visual)
atten_visual_out = tf.keras.layers.Dense(4,activation='softmax')(atten_dense_visual)

# atten_visual_out_re = tf.reshape(atten_visual_out,[tf.shape(atten_visual_out)[0],tf.shape(atten_visual_out)[1],1,-1])

# visual_out_fin = tf.multiply(atten_visual_out_re,stacked_layer_visual)

# visual_out_fin = tf.reshape(visual_out_fin,[tf.shape(visual_out_fin)[0],tf.shape(visual_out_fin)[1],400])

# fused_visual_context_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = True))(visual_out_fin)
  
# visual_fused_pool = tf.keras.layers.GlobalMaxPool1D()(fused_visual_context_layer)



visual_out_fin = tf.multiply (tf.expand_dims(atten_visual_out,2),stacked_layer_visual)
out_visual_fin1 = tf.reshape(visual_out_fin,[tf.shape(visual_out_fin)[0],visual_out_fin.shape[1]*visual_out_fin.shape[2]])
fc_visual=tf.keras.layers.Dense(128,activation='relu')(out_visual_fin1)




# print("video completed")
# ################################################################## audio

X_mfcc=tf.placeholder(tf.float32,[None,emb_dim_mfcc,78])
# mx=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_mfcc)
context_X_mfcc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_mfcc)
fc_X_mfcc=tf.keras.layers.Dense(100,activation='relu')(context_X_mfcc)


X_ege=tf.placeholder(tf.float32,[None,emb_dim_ege,88])
# mx_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_ege)
context_X_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_ege)
fc_X_ege=tf.keras.layers.Dense(100,activation='relu')(context_X_ege)


X_BOAW_ege=tf.placeholder(tf.float32,[None,emb_dim_BOAW_ege,  100])
# mx_BOAW_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_BOAW_ege)
context_X_BOAW_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_BOAW_ege)
fc_X_BOAW_ege=tf.keras.layers.Dense(100,activation='relu')(context_X_BOAW_ege)

X_BOAW_mfcc=tf.placeholder(tf.float32,[None,emb_dim_BOAW_mfcc,100])
# mx_BOAW_mfcc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_BOAW_mfcc)
context_X_BOAW_mfcc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_BOAW_mfcc)
fc_X_BOAW_mfcc=tf.keras.layers.Dense(100,activation='relu')(context_X_BOAW_mfcc)

# stacked_layer_audio=tf.stack([mx,mx_ege,mx_BOAW_mfcc,mx_BOAW_ege],axis=-1)#bs,100,5
concat_layer_audio=tf.keras.layers.Concatenate()([context_X_mfcc,context_X_ege, context_X_BOAW_mfcc,context_X_BOAW_ege])
stacked_layer_audio=tf.stack([context_X_mfcc,context_X_ege, context_X_BOAW_mfcc,context_X_BOAW_ege],axis=1)#bs,100,5
# # concat_layer_audio=tf.keras.layers.Concatenate()([context_X_mfcc,context_X_BOAW_mfcc])

atten_audio_dense=tf.keras.layers.Dense(128,activation='relu')(concat_layer_audio)
atten_audio_out=tf.keras.layers.Dense(4,activation='softmax')(atten_audio_dense)#bs,5

out_audio_fin = tf.multiply (tf.expand_dims(atten_audio_out,2),stacked_layer_audio)
out_audio_fin1 = tf.reshape(out_audio_fin,[tf.shape(out_audio_fin)[0],out_audio_fin.shape[1]*out_audio_fin.shape[2]])
fc_audio=tf.keras.layers.Dense(128,activation='relu')(out_audio_fin1)

# print("audio done")
# ############################################################################# asr

X_asr=tf.placeholder(tf.float32,[None,emb_dim_asr,512])
# mx_asr=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=True))(X_asr)
# context_X_asr=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=True))(mx_asr)
# context_X_asr=tf.reduce_sum(context_X_asr,1)
lstms_fw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [200,200]]
lstms_bw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [200,200]]
#     print('fw',lstms_fw)
drops_fw = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1.0,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_fw]
drops_bw= [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1.0,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_bw]

#      (output_fw, output_bw),final_state =tf.nn.bidirectional_dynamic_rnn(drops_fw,drops_bw,xx)
lstm_output,output_fw, output_bw =tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw=drops_fw,
                                                                                 cells_bw=drops_bw,
                                                                                 inputs=X_asr,dtype=tf.float32)
print('lstm_mfcc',lstm_output.shape)
#     print(output_fw)
#     fs=lstm_output[:, -1]
fs=tf.reduce_sum(lstm_output,1)
fc_X_asr=tf.keras.layers.Dense(128,activation='relu')(fs)
# print("transcript done")
#####################################################FUSION LAYER

fused_stacked_layer=tf.stack([fc_audio,fc_visual,fc_X_asr],axis=1)
fused_concat_layer=tf.keras.layers.Concatenate()([fc_audio,fc_visual,fc_X_asr])
print(fused_concat_layer.shape)  
print(fused_stacked_layer.shape)
fused_atten_dense=tf.keras.layers.Dense(128,activation='relu')(fused_concat_layer)
fused_atten_out=tf.keras.layers.Dense(3,activation='softmax')(fused_atten_dense)
print("ATTENTION SHAPE = ",fused_atten_out.shape)

#PRESET PRIORITY TO ATTENTON
preset = tf.Variable([0,0,1],dtype=tf.float32)
new_atten = tf.nn.softmax(tf.multiply(fused_atten_out,preset))
print("PRESET ATTENTION SHAPE = ",fused_atten_out.shape)


# fused_out_fin = tf.multiply(tf.expand_dims(fused_atten_out,2),fused_stacked_layer)
fused_out_fin = tf.multiply(tf.expand_dims(new_atten,2),fused_stacked_layer)
print(fused_out_fin.shape)
fused_out_fin = tf.reshape(fused_out_fin,[tf.shape(fused_out_fin)[0],fused_out_fin.shape[1]*fused_out_fin.shape[2]])
print(fused_out_fin.shape)
fused_fc=tf.keras.layers.Dense(128,activation='relu')(fused_out_fin)
# fused_fc=tf.keras.layers.Dense(128,activation='relu')(fused_)
fused_fc=tf.keras.layers.Dense(64,activation='relu')(fused_fc)
fused_logits=tf.keras.layers.Dense(1)(fused_fc)
# fusion_inp=tf.concat([fc_audio,fc_visual,fc_X_asr], 0)
# # fusion_inp=tf.reshape(fusion_inp)
# print(fusion_inp.shape)
# fc=tf.keras.layers.Dense(128,activation='relu')(fusion_inp)
# fc=tf.keras.layers.Dense(60,activation='relu')(fc)
# fused_logits=tf.keras.layers.Dense(1)(fc)

 ##############################################################################################
print("fusion done")

loss=tf.reduce_mean(tf.losses.mean_squared_error(labels=true_targets,predictions=fused_logits))
optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(loss)
print('Graph ready')

network started
lstm_mfcc (?, 400, 400)
(?, 384)
(?, 3, 128)
ATTENTION SHAPE =  (?, 3)
PRESET ATTENTION SHAPE =  (?, 3)
(?, 3, 128)
(?, 384)
fusion done
Graph ready


In [72]:
# sess=tf.Session()


In [73]:

saver = tf.train.Saver()

# path = "fused_test/model.ckpt"

path = "fused_test_preset_atten_0/model.ckpt"

In [ ]:
for _ in range(23):
#     tf.reset_default_graph()
    EPOCHS = 40
    BATCH_SIZE = 16

    prev_dev_loss = 200
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:

      try:
        saver.restore(sess,path)
        dev_loss = sess.run(loss,feed_dict={input_bow:dev_seq_bow,input_pose:dev_seq_pose,input_gaze:dev_seq_gaze, input_au:dev_seq_au,
          X_mfcc:val_data_mfcc,X_BOAW_mfcc:val_data_BOAW_mfcc,
          X_BOAW_ege:val_data_BOAW_ege,X_ege:val_data_ege,
          X_asr:val_data_asr,
          true_targets: dev_target})
        print('Dev Loss Currently : ',dev_loss)
        prev_dev_loss = dev_loss
      except:
        print('No pre-saved weights found')
        sess.run(tf.global_variables_initializer())

      for ep in range(EPOCHS):

        num_bts = 0
        ep_loss = 0
        tr_losses = []
        total_aw = []
        print("Epoch:",ep)
        print('Starting.. ', end = ' ')
        train_seq_pose1,train_seq_gaze1,train_seq_au1,train_seq_bow1,train_data_mfcc1,train_data_ege1,train_data_BOAW_mfcc1,train_data_BOAW_ege1,train_data_asr1,target1=shuffle(train_seq_pose,train_seq_gaze,
                                        train_seq_au,train_seq_bow,
                                        train_data_mfcc,train_data_ege,
                                        train_data_BOAW_mfcc,train_data_BOAW_ege,
                                        train_data_asr,target)
        for j in range(len(target)//BATCH_SIZE):
    #       tr_feat = train_ seq_feat[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_pose = train_seq_pose1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_gaze = train_seq_gaze1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_au = train_seq_au1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_bovw=train_seq_bow1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]

          tr_audio_mfcc = train_data_mfcc1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_audio_ege=train_data_ege1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_BOAW_mfcc=train_data_BOAW_mfcc1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_BOAW_ege=train_data_BOAW_ege1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_asr=train_data_asr1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
    #       print(tr_audio_mfcc.shape)
    #       print(tr_pose.shape,tr_gaze.shape,tr_au.shape,tr_bovw.shape)  

          lb = target1[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
    #       print(lb.shape)  
          _,l,aw = sess.run([optimizer,loss,new_atten],feed_dict={input_bow:tr_bovw,input_pose:tr_pose, input_gaze:tr_gaze, input_au:tr_au,
                                                      X_mfcc:tr_audio_mfcc,X_BOAW_mfcc:tr_BOAW_mfcc,
                                                      X_BOAW_ege:tr_BOAW_ege,
                                                      X_ege:tr_audio_ege,
                                                      X_asr:tr_asr,
                                                      lr:0.0001,
                                                      true_targets: lb}) 
          batch_aw = np.mean(aw,axis = 0)
          print("Loss , Attention Wts",l,batch_aw,'==',end = '')
          num_bts+=1
          ep_loss+=l
          total_aw.append(batch_aw)
        print("=")
        ep_loss = ep_loss/num_bts
        print("Loss in epoch:{} = {}".format(ep,ep_loss))
        print("Aggregate Attention Score : ",np.mean(total_aw,axis=0))
        dev_loss = sess.run(loss,feed_dict={input_bow:dev_seq_bow,input_pose:dev_seq_pose,
                                            input_gaze:dev_seq_gaze, input_au:dev_seq_au,
          X_mfcc:val_data_mfcc,X_BOAW_mfcc:val_data_BOAW_mfcc,X_BOAW_ege:val_data_BOAW_ege,X_ege:val_data_ege,
          X_asr:val_data_asr,
          true_targets: dev_target})
        print("Dev Loss in epoch:{} = {}".format(ep,dev_loss))
        tr_losses.append(ep_loss)
        if dev_loss<prev_dev_loss:
          print('SAVING..')
          save_path = saver.save(sess,path) 
          prev_dev_loss = dev_loss

 

Dev Loss Currently :  30.389204
Epoch: 0
Starting..  Loss , Attention Wts 58.625366 [0.21203288 0.21203282 0.57593423] ==Loss , Attention Wts 50.110683 [0.21197546 0.21197541 0.57604915] ==Loss , Attention Wts 32.80461 [0.21216086 0.21216072 0.5756784 ] ==Loss , Attention Wts 31.907757 [0.21201672 0.21201664 0.5759666 ] ==Loss , Attention Wts 24.779053 [0.21205956 0.21205947 0.575881  ] ==Loss , Attention Wts 35.531395 [0.21216407 0.21216395 0.575672  ] ==Loss , Attention Wts 27.225597 [0.21202439 0.2120243  0.57595134] ==Loss , Attention Wts 27.346462 [0.21202706 0.212027   0.5759459 ] ==Loss , Attention Wts 20.199947 [0.21195011 0.21195008 0.5760998 ] ==Loss , Attention Wts 27.775198 [0.21193826 0.21193825 0.5761234 ] ===
Loss in epoch:0 = 33.63060703277588
Aggregate Attention Score :  [0.21203494 0.21203485 0.5759302 ]
Dev Loss in epoch:0 = 30.5555477142334
Epoch: 1
Starting..  Loss , Attention Wts 32.0906 [0.21199062 0.21199057 0.5760188 ] ==Loss , Attention Wts 16.542614 [0.211992

Starting..  Loss , Attention Wts 30.026382 [0.2151767  0.21517344 0.56964993] ==Loss , Attention Wts 22.599163 [0.21584702 0.21584326 0.56830966] ==Loss , Attention Wts 31.962543 [0.21329671 0.2132951  0.5734081 ] ==Loss , Attention Wts 21.487116 [0.21337768 0.21337609 0.5732462 ] ==Loss , Attention Wts 16.14068 [0.21488377 0.21488087 0.5702353 ] ==Loss , Attention Wts 29.623764 [0.21250966 0.21250905 0.5749812 ] ==Loss , Attention Wts 14.649674 [0.21239395 0.2123936  0.5752124 ] ==Loss , Attention Wts 35.887215 [0.21200076 0.21200065 0.5759986 ] ==Loss , Attention Wts 22.047646 [0.21220273 0.21220246 0.5755949 ] ==Loss , Attention Wts 22.888737 [0.21226148 0.21226119 0.5754773 ] ===
Loss in epoch:9 = 24.731292057037354
Aggregate Attention Score :  [0.21339504 0.21339357 0.5732113 ]
Dev Loss in epoch:9 = 24.97468376159668
Epoch: 10
Starting..  Loss , Attention Wts 18.188835 [0.21213411 0.21213391 0.57573193] ==Loss , Attention Wts 14.062922 [0.21278341 0.21278267 0.5744339 ] ==Loss , A

Starting..  Loss , Attention Wts 11.724892 [0.21263306 0.21263228 0.5747347 ] ==Loss , Attention Wts 22.291382 [0.21279371 0.21279271 0.5744136 ] ==Loss , Attention Wts 33.353817 [0.21212496 0.21212462 0.5757504 ] ==Loss , Attention Wts 17.513258 [0.21289477 0.21289366 0.57421154] ==Loss , Attention Wts 15.807699 [0.21307418 0.21307288 0.5738529 ] ==Loss , Attention Wts 10.822003 [0.21445104 0.21444854 0.5711004 ] ==Loss , Attention Wts 32.476593 [0.21355787 0.21355595 0.57288617] ==Loss , Attention Wts 34.295704 [0.21526353 0.21526042 0.56947607] ==Loss , Attention Wts 12.767328 [0.21269172 0.21269077 0.5746176 ] ==Loss , Attention Wts 17.584375 [0.21391259 0.2139107  0.5721767 ] ===
Loss in epoch:18 = 20.863705158233643
Aggregate Attention Score :  [0.21333973 0.21333823 0.573322  ]
Dev Loss in epoch:18 = 20.85788917541504
Epoch: 19
Starting..  Loss , Attention Wts 18.58035 [0.2125565  0.21255575 0.5748877 ] ==Loss , Attention Wts 13.233468 [0.21246466 0.21246387 0.5750716 ] ==Loss ,

In [48]:
a = tf.placeholder(tf.float32, shape=(None,3,3))
b = tf.placeholder(tf.float32, shape=(None,3))
atten = tf.constant([0,0,1],dtype=tf.float32)
atten_out = tf.multiply(b,atten)
print(a.shape)
print(b.shape)
fused_out_fin = tf.multiply (a,tf.expand_dims(atten_out,2))
fc = tf.reshape(fused_out_fin,[tf.shape(fused_out_fin)[0],fused_out_fin.shape[1]*fused_out_fin.shape[2]])
print(fused_out_fin.shape)
print(dot_a_b.shape)
with tf.Session() as sess:
    x = sess.run([fc,fused_out_fin],feed_dict={a: [[[1, 2,4], [3,4,4 ],[5,6,7]],[[1, 1,2], [1,1,2],[4,7,8]]], b: [[2,6,5],[3,9,3]]})

(?, 3, 3)
(?, 3)
(?, 3, 3)
(?, 3, 3)


In [49]:
x[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0., 25., 30., 35.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 12., 21., 24.]], dtype=float32)

In [50]:
x[1]

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [25., 30., 35.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [12., 21., 24.]]], dtype=float32)

In [68]:
a= [[0,0,1],[2,3,2]]
np.mean(a,axis = 0)

array([1. , 1.5, 1.5])